In [1]:
using QuantumStates

The Hamiltonian is assumed to take the following form:
$$
H(A^2\Pi) = T + A \Lambda \Sigma + BN^2 - DN^4 + \frac{1}{2} (p + 2q)(e^{2i\phi} S_- J_- + e^{-2i\phi} S_+ J_+) - \frac{1}{2} q (e^{2i\phi} J_-^2 + e^{-2i\phi} J_+^2).
$$
These terms correspond to the origin, spin-orbit, rotation and centrifugal distortion correction, and $\Lambda$-doubling terms.

In [2]:
H_operator = :(
    T_A * DiagonalOperator +
    Be_A * Rotation + 
    Aso_A * SpinOrbit + 
    q_A * (ΛDoubling_q + 2ΛDoubling_p2q) +
    p_A * ΛDoubling_p2q + 
    # B_z * Zeeman_L +
    # b00_A * Hyperfine_IL +
    # b00_A * Hyperfine_IF +
    b00_A * (Hyperfine_IF - Hyperfine_IL)
);

In [3]:
# Spectroscopic constants for the A²Π state in CaF (from Eunmi Chae's thesis, 2016)
parameters = @params begin
    T_A = 16526.750 * c * 1e2
    Be_A = 0.348781 * c * 1e2
    Aso_A = 71.429 * c * 1e2
    b00_A = 1e6
    p_A = -0.044517 * c * 1e2
    q_A = -2.916e-4 * c * 1e2
end

ParameterList(Dict{Symbol, ComplexF64}(:b00_A => 1.0e6 + 0.0im, :Be_A => 1.04561913293698e10 + 0.0im, :q_A => -8.74194807528e6 + 0.0im, :Aso_A => 2.1413875482482002e12 + 0.0im, :p_A => -1.3345860852786e9 + 0.0im, :T_A => 4.9545950052515e14 + 0.0im))

In [4]:
QN_bounds = (label = "A", S = 1/2, I = 1/2, Λ = (-1,1), J = 1/2:11/2, v = 0)
basis = order_basis_by_m(enumerate_states(HundsCaseA_Rot, QN_bounds))
;

In [5]:
CaF_A_v0 = @time Hamiltonian(basis=basis, operator=H_operator, parameters=parameters)
@time full_evaluate!(CaF_A_v0)
@time solve!(CaF_A_v0)
;

  1.842700 seconds (19.05 M allocations: 636.968 MiB, 14.38% gc time, 48.00% compilation time)
  1.628794 seconds (17.55 M allocations: 530.226 MiB, 34.70% gc time, 21.09% compilation time)
  0.357962 seconds (555.55 k allocations: 33.480 MiB, 93.62% compilation time)


In [6]:
@time save_to_file(CaF_A_v0, "CaF_A_v0", "")

  0.639439 seconds (1.53 M allocations: 75.102 MiB, 96.76% compilation time)


# Save a version with case B as well

In [7]:
QN_bounds = (
    label = "A",
    S = 1/2, 
    I = 1/2, 
    Λ = (-1,1),
    N = 0:6,
    J = 1/2:11/2,
    v = 0
)
CaF_A_v0_caseB_basis = order_basis_by_m(enumerate_states(HundsCaseB_Rot, QN_bounds))
CaF_A_v0_caseB = convert_basis(CaF_A_v0, CaF_A_v0_caseB_basis)

@time save_to_file(CaF_A_v0_caseB, "CaF_A_v0_caseB", "")